In [226]:
import pandas as pd
import pandas as pd
import numpy as np
# from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
## immport desision trees
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,SelectPercentile, chi2


# import seaborn as sns
# import matplotlib.pyplot as plt
# from lazypredict.Supervised import LazyClassifier


import re



In [227]:
data = pd.read_excel(r'Datasets\final_project.ods', engine='odf', dtype='str')
data.head()

,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,"Houston, TX","Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,"Seattle, WA","Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,"Dallas, TX",We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),"Austin, TX",Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter


In [228]:
data['career_level'].value_counts()

career_level
senior_specialist_or_project_manager      4338
manager_team_leader                       2672
bereichsleiter                             960
director_business_unit_leader               70
specialist                                  30
managing_director_small_medium_company       4
Name: count, dtype: int64

In [229]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         8074 non-null   object
 1   location      8074 non-null   object
 2   description   8073 non-null   object
 3   function      8074 non-null   object
 4   industry      8074 non-null   object
 5   career_level  8074 non-null   object
dtypes: object(6)
memory usage: 378.6+ KB


In [230]:
data.dropna(how='any', inplace=True, axis=0)

In [231]:
# df_test = pd.DataFrame(vertorizer.vocabulary_.items(), columns=['key', 'count']).sort_values('count', ascending=False).reset_index(drop=True)
# df_test.head()

In [232]:
def extract_state(x):
    result = re.findall(r'\,\s[A-Z]{2}', x)
    if len(result) > 0:
        return result[0][2:]
    else:
        return x

In [233]:
data['location_state'] = data['location'].apply(extract_state)
data.drop('location', axis=1, inplace=True)
data.head()

,title,description,function,industry,career_level,location_state
0,Technical Professional Lead - Process,"Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager,TX
1,Cnslt - Systems Eng- Midrange 1,"Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager,WA
2,SharePoint Developers and Solution Architects,We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager,TX
3,Business Information Services - Strategic Acco...,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager,North Carolina
4,Strategic Development Director (procurement),Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter,TX


In [234]:
data['career_level'].nunique()

6

In [235]:
for i in data.columns:
    print(i,":" , data[i].nunique())

title : 6789
description : 7973
function : 19
industry : 352
career_level : 6
location_state : 97


In [236]:
target = 'career_level'
x = data.drop(target, axis=1)
y = data[target]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [237]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(6458, 5) (1615, 5) (6458,) (1615,)


In [238]:
x_train.columns

Index(['title', 'description', 'function', 'industry', 'location_state'], dtype='object')

In [239]:
preprocessor = ColumnTransformer(
    transformers=[
        ('title', TfidfVectorizer(), 'title'),
        ('description', TfidfVectorizer(stop_words='english',min_df=0.01, max_df=0.95), 'description'),
        ('function', OneHotEncoder(handle_unknown='ignore'),['function']),
        ('industry', OneHotEncoder(handle_unknown='ignore'),['industry']),
        ('location_state', OneHotEncoder(handle_unknown='ignore'),['location_state'])
    ]
)

In [240]:
# x_train = preprocessor.fit_transform(x_train)
# x_test = preprocessor.transform(x_test)
# x_train.shape, x_test.shape

In [241]:
# cls = Pipeline([
#     ('preprocessor', preprocessor),
#     ('classifier', DecisionTreeClassifier())
# ])

# cls.fit(x_train, y_train)
# y_pred = cls.predict(x_test)
# print(classification_report(y_test, y_pred))
# ### dont use min max df accuracy: 0.68      
# ### use min max df accuracy: 0.69            

In [242]:
# model = SVC()
# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)
# accuracy_score(y_test, y_pred)
# 0.7690402476780186

# cls = Pipeline([
#     ('preprocessor', preprocessor),
#     ('classifier', SVC())
# ])

# cls.fit(x_train, y_train)
# y_pred = cls.predict(x_test)
# print(classification_report(y_test, y_pred))
# # 0.77 , 2 min runs

In [243]:
# model = LogisticRegression()
# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)
# accuracy_score(y_test, y_pred)
# 0.7721362229102167

cls = Pipeline([
    ('preprocessor', preprocessor),
    ("feature_selection", SelectPercentile(chi2, percentile=10)), 
    ('classifier', LogisticRegression())
])

cls.fit(x_train, y_train)
y_pred = cls.predict(x_test)
print(classification_report(y_test, y_pred))

## 0.76, run time: 5 sec

                                      precision    recall  f1-score   support

                      bereichsleiter       0.58      0.37      0.45       182
       director_business_unit_leader       1.00      0.17      0.29        18
                 manager_team_leader       0.69      0.76      0.73       539
senior_specialist_or_project_manager       0.86      0.89      0.87       869
                          specialist       0.00      0.00      0.00         7

                            accuracy                           0.78      1615
                           macro avg       0.63      0.44      0.47      1615
                        weighted avg       0.77      0.78      0.77      1615



d:\Hutson\learning-materials\AI_ML\AIMLDLCV_advance\Class\sklearn-env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Hutson\learning-materials\AI_ML\AIMLDLCV_advance\Class\sklearn-env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Hutson\learning-materials\AI_ML\AIMLDLCV_advance\Class\sklearn-env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [244]:
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(x_train, x_test, y_train, y_test)
# models